Aufgabe 2

In [10]:
import glob
from skimage.io import imread
from skimage import color, img_as_ubyte
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


testImg = []
trainImg = []

testLabel = []
trainLabel = []

# Laden der Test-Bilder und Labels
for img, label in zip(glob.glob('./haribo1/haribo1/hariboTest/*.png'), glob.glob('./haribo1/haribo1/hariboTest/*.png')):
    testImg.append(imread(img))

    parts = label.split('_')
    if label.find('baeren') != -1:
        testLabel.append(0)
    elif label.find('konfekt') != -1:
        testLabel.append(1)
    elif label.find('vampire') != -1:
        testLabel.append(2)

# Laden der Trainings-Bilder und Labels
for img, label in zip(glob.glob('./haribo1/haribo1/hariboTrain/*.png'), glob.glob('./haribo1/haribo1/hariboTrain/*.png')):
    trainImg.append(imread(img))

    parts = label.split('_')
    if label.find('baeren') != -1:
        trainLabel.append(0)
    elif label.find('konfekt') != -1:
        trainLabel.append(1)
    elif label.find('vampire') != -1:
        trainLabel.append(2)


# Funktion zur Berechnung des normierten Farbhistogramms
def normalized_color_histogram(image_channel, bins):
    #image_channel = image_channel.astype(float) #Elementtyp ändern
    histogram, _ = np.histogram(image_channel, bins=bins, range=(0, 256), density=False)
    normalized_histogram = histogram / np.sum(histogram) #Normalisierung
    return normalized_histogram

bins = 4 #vier Bins
trainImgHist = []
testImgHist = []

for i in range(len(trainImg)):
        # Farbhistogramm von jedem Kanal
        red_hist = normalized_color_histogram(trainImg[i][:, :, 0], bins=bins)
        green_hist = normalized_color_histogram(trainImg[i][:, :, 1], bins=bins)
        blue_hist = normalized_color_histogram(trainImg[i][:, :, 2], bins=bins)

        # zu einem 1D Vektor verbinden (hstack zur horizontalen Konkatenation von Arrays)
        feature_vector = np.hstack([red_hist, green_hist, blue_hist])
        trainImgHist.append(feature_vector)
        
        # Ausgabe der Histogramme für Trainingsdaten
        #print(f'Trainingsbild {i + 1} Histogramm: {feature_vector}')

    # Berechnung Farbhistogramme Testdaten
for i in range(len(testImg)):
        # Farbhistogramm von jedem Kanal
        red_hist = normalized_color_histogram(testImg[i][:, :, 0], bins=bins)
        green_hist = normalized_color_histogram(testImg[i][:, :, 1], bins=bins)
        blue_hist = normalized_color_histogram(testImg[i][:, :, 2], bins=bins)

        # zu einem 1D Vektor verbinden (hstack zur horizontalen Konkatenation von Arrays)
        feature_vector = np.hstack([red_hist, green_hist, blue_hist])
        testImgHist.append(feature_vector)

        # Ausgabe der Histogramme für Trainingsdaten
        #print(f'Testsbild {i + 1} Histogramm: {feature_vector}')
trainImgHist

[array([4.34354167e-02, 5.99895833e-02, 8.96410417e-01, 1.64583333e-04,
        5.32083333e-02, 5.16208333e-02, 8.95087500e-01, 8.33333333e-05,
        1.03475000e-01, 2.02483333e-01, 6.94039583e-01, 2.08333333e-06]),
 array([5.63166667e-02, 6.33375000e-02, 8.80331250e-01, 1.45833333e-05,
        8.98291667e-02, 3.40812500e-02, 8.76085417e-01, 4.16666667e-06,
        1.20441667e-01, 8.79541667e-01, 1.66666667e-05, 0.00000000e+00]),
 array([4.16666667e-06, 2.58625000e-02, 9.74079167e-01, 5.41666667e-05,
        5.36458333e-03, 8.27041667e-02, 9.11931250e-01, 0.00000000e+00,
        8.88854167e-02, 9.09877083e-01, 1.23750000e-03, 0.00000000e+00]),
 array([0.00000000e+00, 1.52416667e-02, 9.83395833e-01, 1.36250000e-03,
        4.43541667e-03, 6.05229167e-02, 9.35041667e-01, 0.00000000e+00,
        5.20458333e-02, 9.47904167e-01, 5.00000000e-05, 0.00000000e+00]),
 array([0.00000000e+00, 2.22687500e-02, 9.74579167e-01, 3.15208333e-03,
        1.03125000e-03, 6.25020833e-02, 9.36100000e-01, 

In [8]:
#trainImgHist

In [12]:
X_train_combined = []
X_test_combined = []

for hist in trainImgHist:
    hist1 = hist[:4] #Rot Hist
    hist2 = hist[4:8] #Grün Hist
    hist3 = hist[8:] #Blau Hist
    
    combined_feature_vector = np.hstack([hist1, hist2, hist3]) #Zusammenfassen in Vektor (4+4+4 = 12)
    X_train_combined.append(combined_feature_vector) #hinzufügen zur liste
    
for hist in testImgHist:
    hist1 = hist[:4] #Rot Hist
    hist2 = hist[4:8] #Grün Hist
    hist3 = hist[8:]  #Blau Hist
    
    combined_feature_vector = np.hstack([hist1, hist2, hist3])
    X_test_combined.append(combined_feature_vector)



# One-Hot-Encoding für Trainingsdaten
y_train_one_hot = to_categorical(trainLabel)

# One-Hot-Encoding für Testdaten
y_test_one_hot = to_categorical(testLabel)

#Umwandeln in Arry
X_train_combined = np.array(X_train_combined) 
X_test_combined = np.array(X_test_combined)


In [13]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(12,)))  # input_shape auf 12 ändern
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Ändern auf 3, da du drei Klassen hast

# Modell kompilieren
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Kompiliere das Modell
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trainiere das Modell für 50 Epochen mit einer Batch Size von 1
model.fit(X_train_combined, y_train_one_hot, epochs=50, batch_size=1)

# Evaluieren auf Testdaten
loss, accuracy = model.evaluate(X_test_combined, y_test_one_hot)
print(f'Genauigkeit auf Testdaten: {accuracy * 100:.2f}%')


Epoch 1/50
39/39 [==============================] - 1s 3ms/step - loss: 1.1155 - accuracy: 0.2308
Epoch 2/50
39/39 [==============================] - 0s 3ms/step - loss: 1.0656 - accuracy: 0.4359
Epoch 3/50
39/39 [==============================] - 0s 3ms/step - loss: 1.0617 - accuracy: 0.3846
Epoch 4/50
39/39 [==============================] - 0s 3ms/step - loss: 1.0360 - accuracy: 0.4872
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 1.0293 - accuracy: 0.3333
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 1.0154 - accuracy: 0.5128
Epoch 7/50
39/39 [==============================] - 0s 3ms/step - loss: 1.0132 - accuracy: 0.5385
Epoch 8/50
39/39 [==============================] - 0s 2ms/step - loss: 1.0080 - accuracy: 0.4872
Epoch 9/50
39/39 [==============================] - 0s 2ms/step - loss: 0.9699 - accuracy: 0.5128
Epoch 10/50
39/39 [==============================] - 0s 2ms/step - loss: 0.9566 - accuracy: 0.4872
Epoch 11/50
39/39 [

Aufgabe 3

In [7]:
#CIFAR-10 Daten herunterladen
from tensorflow.keras.datasets.cifar10 import load_data
(x_train, y_train), (x_test, y_test) = load_data()

# Berechnung kanalweise Mittelwert und kanalweise Standardabweichung
mean_train = np.mean(x_train, axis=(1, 2))
std_train = np.std(x_train, axis=(1, 2))

mean_test = np.mean(x_test, axis=(1, 2))
std_test = np.std(x_test, axis=(1, 2))

# Normierung
mean_test = mean_test / 255.0
std_test = std_test / 128.0

mean_train = mean_train / 255.0
std_train = std_train / 128.0

# Erzeugen von Arrays
features_train = np.hstack([mean_train, std_train])
features_test = np.hstack([mean_test, std_test])

# Erstellung des Modells
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(6,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Anzahl der Neuronen auf 10 ändern für 10 Klassen

# Modell kompilieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 20 Epochen Batch Size von 32
model.fit(features_train, y_train, epochs=20, batch_size=32)

# Evaluieren auf Testdaten
_, accuracy = model.evaluate(features_test, y_test)
print(f'Genauigkeit auf Testdaten: {accuracy * 100:.2f}%')

Epoch 1/20
1563/1563 [==============================] - 8s 3ms/step - loss: 2.0679 - accuracy: 0.2319
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.9749 - accuracy: 0.2639
Epoch 3/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9499 - accuracy: 0.2766
Epoch 4/20
1563/1563 [==============================] - 4s 2ms/step - loss: 1.9360 - accuracy: 0.2840
Epoch 5/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.9265 - accuracy: 0.2850
Epoch 6/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.9187 - accuracy: 0.2895
Epoch 7/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.9140 - accuracy: 0.2904
Epoch 8/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.9084 - accuracy: 0.2939
Epoch 9/20
1563/1563 [==============================] - 7s 5ms/step - loss: 1.9038 - accuracy: 0.2941
Epoch 10/20
1563/1563 [==============================] - 4s 2ms/step - loss: 1.901